# Třídy (class)
Třídou je jakýkoliv uživatelský typ. Podobně jako vestavěné typy nabízí metody a data (atributy), ovšem můžeme je libovolně definovat.

Třída je definována pomocí klíčového slova `class` a následuje jej její název. Vnitřek třídy je odsazený o jednu úroveň. 


In [ ]:
class MojeTrida:
    pass

## Metody a atributy

Metoda je puze jiný název pro funkci, která je definována uvnitř třídy.
Definice metody musí být uvnitř bloku třídy. (*Pozn. Metody lze do třídy přidat i později, ale není to preferovaný způsob.*)

Atribut je proměnná, která je definována uvnitř třídy. Atributy mohou být jakékoliv typy, včetně funkcí (ale tomu bychom už zase říkaly metoda :-) pro Python je ale všechno objekt).

Běžné metody (**instance method**) se volají na konkrétním objektu. Kromě nich existují i tzv. **metody třídy** a **statické metody**, které zde nebudeme probírat.

Zvláštnost (*Pozn. Ano, je to opravdu divné.*) definice metod (narozdíl od C++, Javy a dalších jazyků) je ta, že první argument metody je objekt, na kterém je metoda volána. Bez toho by metoda vůbec nevěděla, se kterým objektem pracuje! Dle konvence (která se snad nikdy neporušuje) se tento argument nazývá **self**. Při volání metody se pak vynechává a Python jej automaticky doplní.

Nastavení atributu a jeho hodnoty se provádí podobně jako ukládání do proměnné, ale musíme přidat objekt a tečku. (*Pozn. Interně jsou atributy uložené ve slovnících a při přístupu k nim se prochází slovník samotného objektu, jeho třídy, jejích nadřazených tříd, ...*). Atribut daného jména nemusí přitom vůbec existovat, nemusí se nijak deklarovat.

## Konstruktor
Metoda, která inicializuje objekt - zavolá se na prázdném objektu ve chvíli, kdy vytvoříme novou instanci.
Můžeme jej definovat, ale nemusíme - v takovém případě se použije výchozí konstruktor, který jednoduše nedělá nic (zvláštního). Konstruktor se v Pythonu vždy jmenuje **\_\_init\_\_** (dvě podtržítka před i po).

In [ ]:
# ukázka jednoduché třídy MojeTrida
class MojeTrida:
    def __init__(self, muj_parametr):
        self.muj_parametr = muj_parametr # atribut

    def vypis_muj_parametr(self): # metoda
        print("Atribut ma hodnotu: ", self.muj_parametr)
        
objekt_tridy = MojeTrida("Ahoj") # vytvoření objektu pomocí konstruktoru

objekt_tridy.vypis_muj_parametr() # volání metody

print(objekt_tridy.muj_parametr) # přístup k atributu

V jiných jazicích se můžeme setkat s privátními/chraněnými atributy a metodami, které jsou přístupné jen zvnitř třídy. V Pythonu je to trochu jinak. Všechny atributy a metody jsou veřejné, ale jejich názvy jsou konvencí označeny jako privátní. **Všechny, které začínají podtržítkem `_`, jsou privátní** a neměly by být přístupné zvenčí třídy. V Pythonu je to ale jen konvence, která není vůbec kontrolována a můžeme se k něčemu takovému dostat i zvenčí třídy.

## Property (vlastnosti)
Vlastnosti jsou "chytřejší" data. Umožnují vstoupit do procesu čtení nebo nastavování atributu. Hodí se to například tehdy, pokud objekt má několik navzájem závislých parametrů a my je nechceme ukládat nezávisle; pokud chceme kontrolovat, jaká hodnota se ukládá; či pokud chceme s ukládanou nebo čtenou hodnotou ještě něco zajímavého provést (viz příklad pro kruh).

Ze syntaktického hlediska musíme nejdříve definovat metodu, která nese jméno vlastnosti a která tuto vlastnost "čte" (resp. vrací její hodnotu). O řádek výše musíme umístit tzv. *dekorátor* (tento koncept teď nebudeme podrobně vysvětlovat, jen jej pasivně použijeme) **@property**. Chceme-li, můžeme pak vytvořit i metodu pro zápis - ta se musí jmenovat stejně, požadovat jeden argument (ukládaná hodnota) a být uvedena dekorátorem **@*jmenovlastnosti*.setter**. Podobně bychom mohli vytvořit i metodu pro mazání (dekorátor **@*jmenovlastnosti*.deleter**), ale to se již běžně nedělá.

Jakmile máme takto vytvořené vlastnosti, přistupujeme k nim jako k běžným datovým atributům - voláme je bez závorek a přiřazujeme do nich pomocí znaménka "rovná se".

*Pozn. Vlastnosti fungují podobně jako properties v C# či javabeans v Javě. Povšimněte si však, že pro přístup k vlastnostem se používá úplně stejný zápis jako pro přístup k datovým atributům. Pokud tedy budeme chtít někdy změnit chování datového atributu a udělat z něj vlastnost, klient naší třídy to nepozná a nebude muset dělat žádné změny v kódu. Není tedy vhodné přespříliš iniciativně vytvářet triviální vlastnosti, které jen obalují přístup k atributům (jako by se to jistě dělalo v Javě).*

In [ ]:
import math

class Kruh:
    def __init__(self, r):
        self.polomer = r

    @property                           # Chceme definovat vlastnost pro čtení
    def obsah(self):                    # Vypadá jako obyčejná metoda
        return math.pi * self.polomer ** 2

    @obsah.setter                       # Chceme nastavit zápis do dříve definované vlastnosti
    def obsah(self, s):
        print("Měním obsah na {}".format(s))
        self.polomer = math.sqrt(s / math.pi)

    @obsah.deleter
    def obsah(self):
        pass


kruh = Kruh(1)
print(kruh.polomer)    # Normální datový atribut
print(kruh.obsah)      # Property

kruh.obsah = 3                          # Změníme obsah pomocí zapisovatelné vlastnosti
print(kruh.polomer)    # Normální datový atribut
print(kruh.obsah)      # Property


## Podtržítkové konvence
V Pythonu obecně jsou konvence velice silně zakořeněné. Na objektech to je vidět snad nejvíce.

- "Soukromé" atributy (atributem se v Pythonu často rozumí jak data tak metody -- vše je objekt) se pojmenovávají s podtržítkem na začátku, tj. např. `_soukroma_metoda`.
- Atributy se dvěma podtžítky na začátku i na konci mají speciální význam (viz [dokumentace](http://docs.python.org/3/reference/datamodel.html#special-method-names)). Už jsme viděli `__init__`, podíváme se na několik dalších.
    * `__repr__` a `__str__` převádějí objekt na řetězec.
    * `__getattr__` a `__setattr__` slouží pro čtení a ukládání nenalezených atributů.
    * `__call__` se zavolá pokud použijeme objekt jako funkci.
    * `__doc__` obsahuje dokumentaci (docstring).
    * `__dict__` obsahuje slovník se jmenným prostorem objektu.
    * `__class__` vrací třídu objektu (jak jinak než zase jako objekt).
    * ... dále existují speciální funkce pro logické operátory, pro emulaci funkcionality kontejnerů (iterace, položky, řezy), pro aritmetické operace atd.

In [ ]:
# ukázka __getattr__ a __setattr__
class MojeTrida:
    def __init__(self, muj_parametr):
        self.muj_parametr = muj_parametr

    def __getattr__(self, item):
        print("Zadáváte neexistující atribut: ", item)
        return "Ahoj"

    def __setattr__(self, key, value):
        print("Nastavuji hodnotu atributu: ", key, " na hodnotu: ", value)
        object.__setattr__(self, key, value)
        
        
objekt_tridy = MojeTrida("Ahoj")
print(dir(objekt_tridy))
print(objekt_tridy.tento_parametr_neexistuje)
objekt_tridy.definuji_novy_atribut = "Nazdarek"
print(objekt_tridy.definuji_novy_atribut)
print(dir(objekt_tridy))


In [ ]:
# ukázka __str__ a __repr__
# rozdíl mezi __str__ a __repr__ je v tom, že __str__ je volána při převodu objektu na řetězec 
# a __repr__ je volána při výpisu objektu (např. v interaktivním režimu)
class MojeTrida:
    def __init__(self, muj_parametr):
        self.muj_parametr = muj_parametr

    def __str__(self):
        return "Toto je objekt tridy MojeTrida"

    def __repr__(self):
        return "MojeTrida s parametrem: " + str(self.muj_parametr)
    
objekt_tridy = MojeTrida("Ahoj")
print(objekt_tridy)
print(str(objekt_tridy))
print(repr(objekt_tridy))
objekt_tridy

Možnosti formátování f-stringu jsou vestavěné do třídy kterou vypisujeme, konkrétně metoda `__format__`. Tato metoda je volána při použití f-stringu a použití `:`.

In [ ]:
class Bod3D:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def __format__(self, format_spec):
        if format_spec == 'zyx':
            return f"{self.z}, {self.y}, {self.x}"
        elif format_spec == 'yzx':
            return f"{self.y}, {self.z}, {self.x}"
        else:
            return f"{self.x}, {self.y}, {self.z}"
        
bod = Bod3D(1, 2, 3)
print(f"{bod = }")
print(f"{bod:zyx}")
print(f"{bod:yzx}")
print(f"{bod = :}")


In [ ]:
# seznam atributů __dict__
class MojeTrida:
    def __init__(self, muj_parametr1, muj_parametr2):
        self.muj_parametr1 = muj_parametr1
        self.muj_parametr2 = muj_parametr2

    def vypis_moje_parametry(self):
        print("Atributy mají hodnotu: ", self.muj_parametr1, ", ", self.muj_parametr2)

objekt_tridy = MojeTrida("Ahoj", "Sbohem")
print(objekt_tridy.__dict__)

In [ ]:
# ukázka __call__
class MojeTrida:
    def __init__(self, muj_parametr):
        self.muj_parametr = muj_parametr

    def __call__(self, *args, **kwargs):
        print("Voláte objekt tridy MojeTrida")
        
objekt_tridy = MojeTrida("Ahoj")
objekt_tridy()

In [ ]:
# ukázka __doc__
class MojeTrida:
    """Toto je ukázka dokumentace třídy MojeTrida"""
    def __init__(self, muj_parametr):
        self.muj_parametr = muj_parametr

    def vypis_muj_parametr(self):
        """Toto je ukázka dokumentace metody vypis_muj_parametr"""
        print("Atribut ma hodnotu: ", self.muj_parametr)
        
objekt_tridy = MojeTrida("Ahoj")
print(objekt_tridy.__doc__)
print(objekt_tridy.vypis_muj_parametr.__doc__)

Třídy mohou také definovat funkce `__getitem__` a `__setitem__` díky kterým je možné přistupovat k objektu jako k poli/slovníku. Tato chování se pak použije v případě, že se použije operátor `[]` (např. `obj[1]`).

In [ ]:
# ukázka __setitem__ a __getitem__
class MojeTrida:
    def __init__(self, muj_parametr):
        self._moje_data = dict()
        self.muj_parametr = muj_parametr + " "

    def __setitem__(self, key, value):
        print("Nastavuji hodnotu atributu: ", key, " na hodnotu: ", value)
        self._moje_data[key] = self.muj_parametr + str(value)

    def __getitem__(self, item):
        return self._moje_data[item]
    
objekt_tridy = MojeTrida("Ahoj")
objekt_tridy[0] = "Honzo"
objekt_tridy[1] = "Pepo"
objekt_tridy[2] = "Jardo"

print(objekt_tridy[0])
print(objekt_tridy[1])
print(objekt_tridy[2])


### Podtržítkové metody implementující algebraické operace
V Pythonu je možné nadefinovat chování operátorů (+,-,*,/,...) pro vlastní objekty. Toho můžeme docílit tak, že nadefinujeme speciální metody:
- `__add__` pro operátor `+`
- `__sub__` pro operátor `-`
- `__mul__` pro operátor `*`
- `__div__` pro operátor `/`
- `__truediv__` pro operátor `/` (v Pythonu 3 je toto stejné jako `__div__`)
- `__floordiv__` pro operátor `//`
- `__mod__` pro operátor `%`
- `__pow__` pro operátor `**`
- `__lshift__` pro operátor `<<`
- `__rshift__` pro operátor `>>`
- `__and__` pro operátor `&`
- `__xor__` pro operátor `^`
- `__or__` pro operátor `|`
- a další (viz [dokumentace](http://docs.python.org/3/reference/datamodel.html#special-method-names)).

Metody se vždy volají na levém operandu, pravý operand je předán jako argument. Pokud pravý operand není kompatibilní s operací, metoda by měla vyhodit výjimku. Poté se pokusí Python zavolat metodu na pravém operandu s prefixem `r` (tj. pokud se pokusíme sčítat číslo s objektem, Python zavolá `__add__` na čísle a pokud to selže, pokusí se zavolat `__radd__` na objektu).

In [ ]:
class A:
    def __init__(self):
        pass

    def __add__(self, other):
        print("A.__add__")
        return self
    
    def __radd__(self, other):
        print("A.__radd__")
        return self


class B:
    def __init__(self):
        pass

    def __add__(self, other):
        print("B.__add__")
        return self


a = A()
b = B()
c = a + b
c = b + a
c = a + 5
c = 5 + a
c = 5 + b

Ukázka na implementaci komplexních čísel:


In [ ]:
class KomplexniCislo:
    def __init__(self, realna_cast, imaginarni_cast):
        self.realna_cast = realna_cast
        self.imaginarni_cast = imaginarni_cast

    def __add__(self, other):
        return KomplexniCislo(self.realna_cast + other.realna_cast, self.imaginarni_cast + other.imaginarni_cast)

    def __sub__(self, other):
        return KomplexniCislo(self.realna_cast - other.realna_cast, self.imaginarni_cast - other.imaginarni_cast)

    def __mul__(self, other):
        realna = self.realna_cast * other.realna_cast - self.imaginarni_cast * other.imaginarni_cast
        imaginarni = self.imaginarni_cast * other.realna_cast + self.realna_cast * other.imaginarni_cast
        return KomplexniCislo(realna, imaginarni)

    def __truediv__(self, other):
        jmenovatel = other.realna_cast**2 + other.imaginarni_cast**2
        realna = (self.realna_cast * other.realna_cast + self.imaginarni_cast * other.imaginarni_cast) / jmenovatel
        imaginarni = (self.imaginarni_cast * other.realna_cast - self.realna_cast * other.imaginarni_cast) / jmenovatel
        return KomplexniCislo(realna, imaginarni)

    def __str__(self):
        return f"{self.realna_cast} + {self.imaginarni_cast}i"

    def __repr__(self):
        return f"KomplexniCislo({self.realna_cast}, {self.imaginarni_cast})"


komplexni1 = KomplexniCislo(1, 2)
komplexni2 = KomplexniCislo(3, 4)
print(komplexni1 + komplexni2)
print(komplexni1 - komplexni2)
print(komplexni1 * komplexni2)
print(komplexni1 / komplexni2)
komplexni1 / komplexni2

## Dědičnost
Třída může svoje chování (i data) odvozovat od nějaké jiné třídy, čímž si ušetříme spoustu práce při opakování společných rysů. V takovém případě řekneme, že naše nová třída (dceřinná) od té původní (rodičovské) dědí.

* V dceřinné třídě můžeme změnit definici některé metody z rodičovské třídy.
* Konstruktory se standardně dědí (*Na rozdíl od C++ či Javy, kde se musí explicitně volat, v Pythonu se volají jen pokud definujeme nový konstruktor a chceme zavolat i nadřazený.*)
* Instance dceřinné třídy se mohou použít kdekoliv, kde počítá s objektem rodičovské třídy. *Toto platí v Pythonu ještě obecněji - obvykle se nekontrolují konkrétní typy, projde jakýkoliv objekt, který nabízí používané atributy/metody.*

**Syntax:** Jméno rodičovské třídy se dává do závorky za jméno (místo object, od kterého třídy obvykle dědí).

In [ ]:
class Clovek:
    def __init__(self, jmeno):         # Konstruktor, který nastaví atribut "jmeno"
        self.jmeno = jmeno
        
    def _rekni(self, text):            # Privátní metoda, která vypíše text
        print(self.jmeno, ": ", text)

    def predstav_se(self):
        self._rekni("Jmenuji se " + self.jmeno + ".")

    def pozdrav(self):
        self._rekni("Dobrý den.")

    def rozluc_se(self):
        self._rekni("Nashledanou.")


class Elektrikar(Clovek):
    def oprav_televizi(self):         # Nová metoda v rodičovské třídě - jiný Clovek ji neumí
        self._rekni("Bude to v cuku letu.")
        print("---Elektrikar něco šudlá.---")
        self._rekni("A je to.")

    def predstav_se(self):            # Předefinovaná metoda "predstav_se" využívá atribut rodičovské třídy
        self._rekni("Já sem ňákej " + self.jmeno + ".")


class Zakaznik(Clovek):
    def nakupuj(self):                # Nová metoda v rodičovské třídě - jiný Clovek ji neumí
        self._rekni("Prosím opravíte mi, televizi.")


e = Elektrikar("Franta Vopička")
z = Zakaznik("Tomáš Marný")

# Rozhovor
z.pozdrav()          # Všimněte si, že "pozdrav" je rodičovská metoda, ale volá se "rekni" z dceřinné třídy.
e.pozdrav()
z.predstav_se()
e.predstav_se()
z.nakupuj()
e.oprav_televizi()

## Dataclass
Od Pythonu 3.7 je možné používat dekorátor `@dataclass`, který umožňuje jednoduše definovat třídy, které se pouze skládají z datových atributů.

Výhody použití `@dataclass` je automatické generování několika metod:
- `__init__` (konstruktor)
- `__repr__` (reprezentace objektu jako řetězce)
- `__eq__` (rovnost)
- `__lt__` (menší než)
- `__le__` (menší nebo rovno)
- `__gt__` (větší než)
- `__ge__` (větší nebo rovno)
- ... a další (viz [dokumentace](https://docs.python.org/3/library/dataclasses.html#dataclasses.dataclass)).

In [ ]:
from dataclasses import dataclass

@dataclass(order=True)
class Point3D:
    x: float
    y: float
    z: float
    
bod1 = Point3D(1, 2, 3)
bod2 = Point3D(1, 3, 2)

print(bod1)
print(bod1 < bod2) # jak je toto definováno? viz dokumentace


## Další témata (zde se jimi nebudeme zabývat)
Ale jsou to věci, které je užitečné znát, alespoň vědět, že existují.

* Vícenásobná dědičnost
* Metody třídy
* Statické metody
* Abstraktní třídy
* Metatřídy (Dataclass, ...)
* Návrhové vzory
